To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer python-dotenv
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "OpenBuddy/openbuddy-qwen2.5llamaify-7b-v23.1-200k",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/873 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We load and prepare our training dataset here

In [ ]:
import json
from collections import defaultdict

# Load the dialog data
file = "dialog_training_data.json"

with open(file, "r") as f:
    data = json.load(f)

# Prepare a dict to collect pairs by category
category_pairs = defaultdict(list)

for dialog_entry in data:
    translated_dialog = dialog_entry["translated_dialog"]
    category = dialog_entry["category"]

    # Iterate in steps of 2 -> index 0 prompt, 1 response, 2 prompt, 3 response, etc.
    for i in range(0, len(translated_dialog) - 1, 2):
        prompt = translated_dialog[i].strip()
        response = translated_dialog[i + 1].strip()

        category_pairs[category].append({
            "translated_question": prompt,
            "translated_answer": response
        })
        break

# Merge pairs in order of category (0, 1, 2...)
sorted_pairs = []
for category in sorted(category_pairs.keys()):
    sorted_pairs.extend(category_pairs[category])

# Save the pairs to a new JSON file
output_file = "final_training_3k.json"
with open(output_file, "w") as f:
    json.dump(sorted_pairs, f, ensure_ascii=False, indent=2)

print(f"Saved {len(sorted_pairs)} prompt-response pairs sorted by category to {output_file}")


Saved 9000 prompt-response pairs sorted by category to final_training_3k.json


In [ ]:
from datasets import Dataset

EOS_TOKEN = tokenizer.eos_token
custom_prompt = """### Prompt:
{}

### Response:
{}
"""

def get_training_dataset():
    with open("final_training_3k.json", "r") as f:
        data = json.load(f)

    dataset = Dataset.from_list(data)

    def formatting_func(examples):
        prompts = examples["translated_question"]
        responses = examples["translated_answer"]
        texts = []
        for prompt, response in zip(prompts, responses):
            text = custom_prompt.format(prompt, response) + EOS_TOKEN
            texts.append(text)
        return {"text": texts}

    dataset = dataset.map(formatting_func, batched=True)

    return dataset

dataset = get_training_dataset()

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 256,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        save_strategy = "steps",       # Save periodically during training
        save_steps = 20,               # Every 20 steps
        save_total_limit = 2,          # Keep only last 2
        eval_strategy = "no",

    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/9000 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.096 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,000 | Num Epochs = 1 | Total steps = 1,125
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,000,000,000 (0.58% trained)


Step,Training Loss
10,2.444500
20,2.400700
30,2.556200
40,2.272300
50,2.343500
60,2.409600
70,2.406000
80,2.457100
90,2.303200
100,2.396700


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3788.9504 seconds used for training.
63.15 minutes used for training.
Peak reserved memory = 7.096 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 48.138 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    custom_prompt.format(
        "I am beginner in Spanish.",
        ""
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['### Prompt:\nI am beginner in Spanish.\n\n### Response:\n\n¿Cómo es que?\n<|end|>']

<a name="Save"></a>
### Saving finetuned model

We directly save the merged 16 bit version to huggingface.

In [ ]:
from google.colab import drive
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Get HF Token from google drive
drive.mount('/content/drive')
load_dotenv('/content/drive/MyDrive/secrets/.env')
hf_token = os.getenv("HF_TOKEN")

login(token=hf_token)

In [ ]:
model_name = "SpanishLanguageTeacher-7B"

save_to_hf = True
username = "49Simoney"
repo_name = model_name + "-9k-merged"
if save_to_hf:
    model.save_pretrained_merged(model_name, tokenizer, save_method = "merged_16bit",)
    model.push_to_hub_merged(username + "/" + repo_name, tokenizer, save_method = "merged_16bit", token = hf_token)

Unsloth: You are pushing to hub, but you passed your HF username = 49Simoney.
We shall truncate 49Simoney/SpanishLanguageTeacher-7B-9k-merged to SpanishLanguageTeacher-7B-9k-merged
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 15.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.98 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 39%|███▉      | 11/28 [00:00<00:01, 13.36it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [00:56<00:00,  2.00s/it]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

...

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/49Simoney/SpanishLanguageTeacher-7B-9k-merged
